In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

Using TensorFlow backend.


In [2]:
#readind and preparing data ( text , label )
import os, sys
import pandas as pd
import numpy as np
df=pd.read_excel('ml2.xlsx', header=None) ## read le file
df.columns=['tags','post']
print(df[1:5])

   tags                                               post
1    12  محضر تحقيق بالشكوى المقدمة بنك الشرق ضد جوزف ع...
2    13      حول ادعاء طوني يارد ضد مجهول بجرم سرقة محفظته
3    11  محضر تحقيق فوري بتوقيف اللبناني فارس ورده بجرم...
4    12  محضر تحقيق بإدعاء حسام حسين قبيسي ضد المدعى عل...


In [3]:

import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
STOPWORDS = set(stopwords.words('arabic'))
from bs4 import BeautifulSoup
df = df.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9ا-ي #+_]')
STOPWORDS = set(stopwords.words('arabic'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    #text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('x', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df['post'] = df['post'].apply(clean_text)

In [4]:
print(df['post'][1])
table=str.maketrans("+=_-<>~/!*%?:;,)&1234567890@$qwertyuioplkjhgfdsamnbvcxz(QWERTYUIOPLKJHGFDSAZXCVBNM",82*" ")
for i in range(len(df['post'])):
    df['post'][i] = df['post'][i].translate(table)
print(df['post'][1])


محضر تحقيق بالشكوى المقدمة بنك الشرق ضد جوزف عبدو سعد بجرم شك رصيد


C:\Users\Maya\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


محضر تحقيق بالشكوى المقدمة بنك الشرق ضد جوزف عبدو سعد بجرم شك رصيد


In [5]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re

In [6]:
def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(TaggedDocument(v.split(), [label]))
    return labeled

In [7]:
from sklearn.model_selection  import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.post, df.tags, random_state=0, test_size=0.3)
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test

In [8]:

all_data[:2]

[TaggedDocument(words=['تحقيق', 'بادعا', 'رواد', 'ربيع', 'داغر', 'ضد', 'عدد', 'الشخاص', 'بجرم', 'ضرب', 'وايذا'], tags=['Train_0']),
 TaggedDocument(words=['الساعة', 'تاريخه', 'وفي', 'محلة', 'مار', 'الياس', 'امام', 'سنتر', 'المقاصد', 'حصل', 'حادث', 'اصطدام', 'سيارة', 'نوع', 'مرسيدس', 'رقم', 'ج', 'بقيادة', 'ندين', 'خضر', 'الجمل', 'والدتها', 'سوسن', 'تولد', 'لبنانية', 'ودراجة', 'نارية', 'نوع', 'عقاد', 'رقم', 'م', 'بقيادة', 'كامل', 'راشد', 'جعفر', 'والدته', 'محموده', 'تولد', 'لبناني', 'نقل', 'الى', 'مستشفى', 'المقاصد', 'لاصابته', 'برضوض', 'وحالته', 'مستقرة'], tags=['Train_1'])]

In [9]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

C:\Users\Maya\Anaconda3\lib\site-packages\gensim\models\base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "
100%|█████████████████████████████████████████████████████████████████████████| 7242/7242 [00:00<00:00, 1448643.15it/s]


In [10]:
for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|█████████████████████████████████████████████████████████████████████████| 7242/7242 [00:00<00:00, 1806216.90it/s]


In [11]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors

In [12]:
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

In [13]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(train_vectors_dbow, y_train)

C:\Users\Maya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Maya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=1, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [14]:
logreg = logreg.fit(train_vectors_dbow, y_train)
y_pred = logreg.predict(test_vectors_dbow)

C:\Users\Maya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Maya\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [15]:
from sklearn.metrics import accuracy_score, classification_report
print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.8104003681546249
              precision    recall  f1-score   support

           1       0.64      0.68      0.66        93
           2       0.70      0.55      0.62        98
           3       0.88      0.83      0.86       129
           4       0.54      0.60      0.57       115
           5       0.74      0.71      0.73       139
           6       0.56      0.55      0.55       116
           7       0.90      0.84      0.87       139
           8       0.97      0.97      0.97       158
           9       0.78      0.88      0.83       164
          10       0.71      0.72      0.71       200
          11       0.97      0.98      0.98       231
          12       0.94      0.98      0.96       290
          13       0.82      0.78      0.80       301

   micro avg       0.81      0.81      0.81      2173
   macro avg       0.78      0.78      0.78      2173
weighted avg       0.81      0.81      0.81      2173

